In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/' 
sys.path.append(parent_dir) 

import pandas as pd
import numpy as np
from copy import deepcopy

from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time, auto_save_pickle

from models.fmin2 import fmin2
from models.nested_validation import *
from models.features import fixed_makedata, salvador_wrap, pretrained_wrap
from models.display import pickle2df

# models
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

pd.set_option('display.max_colwidth', -1)

import warnings
warnings.filterwarnings("ignore")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.15 µs


In [2]:
dir_save = os.path.normpath('../data/dic_20190819.pickle')
dic = load(dir_save)

amt_GI = load_pickle('../data/amt_hGI_pos.pickle')
for i, v in dic.items():
    if i in amt_GI.keys():
        v['AMT'] = amt_GI[i]
amt_ls = list(amt_GI.keys())
ls = amt_ls

exist
time: 14.6 s


### 1. BoW-NB and LR

In [3]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lr
    
    spaces = {'model1': model1, 'model2': LogisticRegression,'method': method,
              'p2': p2_lr, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 10)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2

'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
model1 = None
state = 2022
for method in [['wc','nb','12']]: # do not scale seems better
    print('state', state)
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2), method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-20 15:41:47.609935
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 0.5}
2019-08-20 15:41:47.875377
[0.7547169811320754, 0.8024691358024691, 0.830188679245283, 0.7901234567901234, 0.7712418300653595]
0.7897480166070621
-0.7897480166070621 -0.7897480166070621
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
2019-08-20 15:42:00.812333
[0.7515923566878981, 0.7901234567901234, 0.8220858895705522, 0.7757575757575756, 0.7564102564102564]
0.7791939070432811
-0.7791939070432811 -0.7897480166070621
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 3.0}
2019-08-20 15:42:13.061992
[0.7468354430379747, 0.75, 0.8242424242424242, 0.7682926829268293, 0.7564102564102564]
0.7691561613234968
-0.7691561613234968 -0.7897480166070621
{'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l2', 'C': 1000}
2019-08-20 15:42:26.104703
[0.7341772151898733, 0.7560975609756098, 0.8170731707317075, 

,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.817,0.844,0.794,0.818,0.938,0.948,0.929,0.938,None,"LogisticRegression(C=0.05, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n solver='liblinear', tol=0.0001, verbose=0, warm_start=False)","[wc, nb, 12]",AMT,../pickle/2019-08-20 15:52:26.137797.pickle


save to ../csv/2019-08-20 15:52:26.255971.csv


,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.817,0.844,0.794,0.818,0.938,0.948,0.929,0.938,None,"LogisticRegression(C=0.05, class_weight='balanced', dual=False,\n fit_intercept=True, intercept_scaling=1, max_iter=100,\n multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n solver='liblinear', tol=0.0001, verbose=0, warm_start=False)","[wc, nb, 12]",AMT,../pickle/2019-08-20 15:52:26.137797.pickle


time: 10min 50s


### 2. BoW-NB and LGBM

In [4]:
def inner_CV(corp, medel1, method):
    '''
    should speficify the parameters here
    '''
    from models.fmin2 import p2_lgbm
    
    spaces = {'model1': model1, 'model2': LGBMClassifier,'method': method,
              'p2': p2_lgbm, 'corp': corp}
    
    ''' fmin from hyperopt
    Args:
        fn: model to run, in the form of function
        space: dict of potential parameters
    Return:
        best: dict of best parameters
    ''' 
    best_space = fmin2(fn = objective, space=spaces, max_evals = 20)
    print(best_space)
    model2 = best_space['model2'](**best_space['p2'])
    return model2

'''
0. start the model selection and evaluation
'''
dfs = []
corp0 = fixed_makedata(dic, ls, tag='AMT')
knowns = corp0.knowns
model1 = None
state = 2022
for method in [['wc','nb']]:
    print('state', state)
    corp = copy.deepcopy(corp0)
    results = []
    model2, fold_count = '', 0
    ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = state)
    for train_index, test_index in ss.split(corp.ls, corp.y):
        print('outer fold')
        print_time()
        '''
        1. model selection
        ::: model1 is for feature engineering
            e.g. model1 = pretrained_wrap(corp.knowns, 'glove-wiki-gigaword-300')
            e.g. model1 = gensim_wrap(corp.knowns, Word2Vec, params = {})
        ::: model2 is a classifier
            e.g. model2 = copy.deepcopy(classifier['default_Logistic'])
        '''
        model2 = inner_CV(corp.replace(train_index), model1, method)
        '''
        2. build X
        '''
        data = inputs_generater(corp.replace(train_index), model1, method)
        '''
        3. run and find the best prob threshold
        '''
        threshold = clf_running_search(data, model1, copy.deepcopy(model2), method)
        '''
        4. run again and evaluate on the real test set
        ''' 
        data = inputs_generater(corp.add_train_test(train_index, test_index), model1, method)
        result = clf_running(data, model1, copy.deepcopy(model2), method, threshold).result
        result.update({'model1': model1,'tag': corp.tag, 'method':method})
        results.append(result)
        print(result['test_f1'])

    pickle_path = auto_save_pickle(results)
    df = pickle2df(results, pickle_path)
    dfs.append(df)
auto_save_csv(pd.concat(dfs))

state 2022
outer fold
2019-08-20 15:52:38.573724
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 256, 'max_depth': 16, 'learning_rate': 0.15, 'gamma': 0.65, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.5, 'bagging_fraction': 0.75, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-20 15:52:38.880770
[0.6962025316455697, 0.7374999999999999, 0.7261146496815287, 0.7719298245614035, 0.7058823529411764]
0.7275258717659356
-0.7275258717659356 -0.7275258717659356
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 512, 'max_depth': 8, 'learning_rate': 0.15, 'gamma': 0.55, 'n_estimators': 2000, 'lambda_l2': 1, 'feature_fraction': 0.75, 'bagging_fraction': 0.5, 'bagging_freq': 5, 'subsample': 0.7}
2019-08-20 16:01:23.560798
[0.675, 0.7307692307692308, 0.7169811320754716, 0.7485380116959064, 0.7250000000000001]
0.7192576749081219
-0.7192576749081219 -0.7275258717659356
{'class_weight': 'balanced', 'boosting': 'gbrt', 'num_leaves': 64, 'max_depth': 8, 'learning_

,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.77,0.774,0.767,0.767,1,1,1,1,None,"LGBMClassifier(bagging_fraction=0.9, bagging_freq=10, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.75, gamma=0.45,\n importance_type='split', lambda_l2=1, learning_rate=0.1,\n max_depth=4, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=64,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=1, subsample_for_bin=200000,\n subsample_freq=0)","[wc, nb]",AMT,../pickle/2019-08-21 23:42:08.928957.pickle


save to ../csv/2019-08-21 23:42:10.497253.csv


,test_f1,test_precision,test_recall,test_accuracy,train_f1,train_precision,train_recall,train_accuracy,model1,model2,method,tag,pickle_path
0,0.77,0.774,0.767,0.767,1,1,1,1,None,"LGBMClassifier(bagging_fraction=0.9, bagging_freq=10, boosting='gbrt',\n boosting_type='gbdt', class_weight='balanced',\n colsample_bytree=1.0, feature_fraction=0.75, gamma=0.45,\n importance_type='split', lambda_l2=1, learning_rate=0.1,\n max_depth=4, min_child_samples=20, min_child_weight=0.001,\n min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=64,\n objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n silent=True, subsample=1, subsample_for_bin=200000,\n subsample_freq=0)","[wc, nb]",AMT,../pickle/2019-08-21 23:42:08.928957.pickle


time: 1d 7h 49min 44s
